In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, binom
from scipy import stats

In [2]:
from src.MM_estimation import calc_variance_of_default_rate
from src.MM_estimation import estimate_w_factor_loading
from src.ML_estimation import calculate_my_likelihood
from src.ML_estimation import calculate_my_likelihood_arr
from src.ML_estimation import maximum_likelihood_estimation
from src.monte_carlo import monte_carlo_MLE, expected_value_of_function_monte_carlo, likelihood_mc
from src.sucess_probability import p_g

In [3]:
df = pd.read_csv('SP_historical_PD_data.csv', sep=';')
df["pd_total"] = df["Default rate (%)"] / 100
df["pd_inv"] = df["Investment-grade default rate (%)"] / 100
df["pd_spec"] = df["Speculative-grade default rate (%)"] / 100
# Calculate the number of obligors
df['num_of_inv_grades'] = (df['Investment-grade defaults'] / (df["pd_inv"])).round()
df['num_of_spec_grades'] = (
            df['Speculative-grade defaults'] / (df["pd_spec"])).round().astype(int)
df['num_of_total_grades'] = (df['Total defaults*'] / (df["pd_total"])).round().astype(int)

# Fill-out the missing values in num_of_inv_grades column with the difference between num_of_total_grades and num_of_spec_grades
df['num_of_inv_grades'] = np.where(df['num_of_inv_grades'].isna(), df['num_of_total_grades'] - df['num_of_spec_grades'],
                                   df['num_of_inv_grades']).astype(int)

In [4]:
# MM for speculative grade
MM_params = estimate_w_factor_loading(df["pd_spec"], df["num_of_spec_grades"], initial_guess=0.27)
print("Estimated parameters for speculative grade: ", MM_params[0], norm.ppf(MM_params[1]))

Estimated parameters for speculative grade:  0.07602889962521624 -1.7498743361043698


In [5]:
# ML for speculative grade
ML_params = maximum_likelihood_estimation(df["Speculative-grade defaults"], df["num_of_spec_grades"], norm.pdf, p_g, 0.27, -2, [(0, 1), (-np.inf, np.inf)])
print("Estimated parameters for speculative grade: ", ML_params)

Estimated parameters for speculative grade:  (0.0, -1.78533136062324)


In [6]:
Monte_carlo_params = monte_carlo_MLE([df["Speculative-grade defaults"].sum()], [df["num_of_spec_grades"].sum()], p_g, [0.27], [-2], [(0, 1), (-np.inf, np.inf)], num_of_simulations=10000)
print("Estimated parameters for speculative grade: ", Monte_carlo_params.x)

Estimated parameters for speculative grade:  [ 0.28771878 -1.72499994]


In [3]:
# An example for the calculation of the integrand
test_d_g_arr = np.array([5, 8])
test_n_g_arr = np.array([50, 100])
prob_dens_func = norm.pdf
test_w_g_arr = np.array([0.9, 0.9])
test_gamma_g_arr = np.array([-1.6449, -1.6449])

calculate_my_likelihood_arr(test_d_g_arr, test_n_g_arr, p_g, prob_dens_func, test_w_g_arr, test_gamma_g_arr)

0.0008893194009710714

In [4]:
test_lamda_func = lambda x: np.prod([stats.binom.pmf(d, n, p_g(x, w_g, gamma_g)) for d, n, w_g, gamma_g in zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, test_gamma_g_arr)], axis = 0)

expected_value_of_function_monte_carlo(test_lamda_func, num_samples=1000000)

0.0008920264232967348

In [5]:
likelihood_mc(test_w_g_arr[0], test_gamma_g_arr, test_d_g_arr, test_n_g_arr, p_g, num_of_simulations=1000000)

0.0008877287473450451

In [10]:
#TO-DO fix this in likelihood_mc function:
"""
list(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, test_gamma_g_arr))
list(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, [-1.96]]))
"""

'\nlist(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, test_gamma_g_arr))\nlist(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, [-1.96]]))\n'

In [11]:
from src.sucess_probability import p_g

In [6]:
from src.data_generator import generate_default_buckets
from scipy.optimize import minimize

In [16]:
#np.random.seed(3123) # Fix the seed for reproducibility
np.random.seed()
time_horizon = 40
factor_loading_list = [0.45, 0.45, 0.45]
num_of_obligors_list = [3000, 3000, 3000]
gamma_list = [-2.9, -2.3, -1.6]
d_g_list = generate_default_buckets(factor_loading_list,num_of_obligors_list, gamma_list,time_horizon)
# multiply all num_of_obligors with time_horizon
n_g_list = [num_of_obligors_list[i] * time_horizon for i in range(len(num_of_obligors_list))]

In [17]:
d_g_list, n_g_list

([359, 1793, 7959], [120000, 120000, 120000])

In [18]:
#SP data
# d_g_list = np.array([64, 234, 1027, 5030, 29903])
# n_g_list = 40 * np.array([3000, 3000, 3000, 3000, 3000])
# 
# d_g_list, n_g_list

In [19]:
from scipy.integrate import quad

def calculate_my_likelihood_arr(d_g_arr, n_g_arr, p_g, prob_dens_func, w_g_arr, gamma_g_arr):
    """
    Numerically calculates the value of L(d_g_arr) for multiple grades based on the given formula.

    Parameters:
        d_g_arr (numpy.array(int)): Values of d_g's by grades
        n_g_arr (numpy.array(int)): Values of n_g's by grades
        p_g (callable): The p_g function representing the probability density function.
        prob_dens_func (callable): The pdf_g function representing the probability density function.
        w_g_arr (numpy.array(float)): Parameter 'w_g's by grades
        gamma_g_arr (numpy.array(float)): Parameter 'gamma_g's by grades.

    Returns:
        float: Numerical approximation of the integral.
    """

    integrand = lambda x: np.prod(binom.pmf(d_g_arr, n_g_arr, p_g(x, w_g_arr, gamma_g_arr))) * prob_dens_func(x)

    result, _ = quad(integrand, -3, 3) # , points=5, epsrel = 1e-012

    return result

In [35]:
def parameter_estimation(default_list, num_of_obligors_over_time, factor_loading_init, gamma_list_init):
    initial_guess = gamma_list_init + factor_loading_init
    
    num_of_gamma = len(gamma_list_init)
    num_of_factor_loading = len(factor_loading_init)
    # bound = num_of_gamma * (-5, 5) + num_of_factor_loading * (-1, 1)
    bounds = num_of_gamma * [(-5, 5)] + num_of_factor_loading * [(-1, 1)]
    # print(bounds)
    # print(initial_guess)
    # Optimization
    objective_function = lambda params: -np.log(calculate_my_likelihood_arr(
        default_list, num_of_obligors_over_time, p_g, norm.pdf, params[num_of_gamma], params[0:num_of_gamma]
    ))
    
    result = minimize(objective_function,
                  initial_guess,
                  method="Nelder-Mead",
                  bounds=bounds,
                  options={
                      'disp': False})
    
    return result

In [36]:
conf_int_80 = {}
input_80 = {}
results_dict_80 = {}

np.random.seed(42)
time_horizon = 80
factor_loading_list = [0.45, 0.45, 0.45]
num_of_obligors_list = [3000, 3000, 3000]
gamma_list = [-2.9, -2.3, -1.6]
factor_loading_init = [0.447]
gamma_list_init = [-2.91, -2.29, -1.603]

for i in range(100):
    d_g_list = generate_default_buckets(factor_loading_list,num_of_obligors_list, gamma_list,time_horizon)
    n_g_list = [num_of_obligors_list[i] * time_horizon for i in range(len(num_of_obligors_list))]
    ML_params = parameter_estimation(d_g_list, n_g_list, factor_loading_list, gamma_list)
    input_80[i] = (d_g_list, n_g_list)
    results_dict_80[i] = ML_params.x
    conf_int_80[i] = ML_params.x[3]

C:\Users\kissp\AppData\Local\Temp\ipykernel_28604\873326530.py:11: RuntimeWarning: divide by zero encountered in log
  objective_function = lambda params: -np.log(calculate_my_likelihood_arr(
C:\Users\kissp\AppData\Local\Temp\ipykernel_28604\1971462210.py:21: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = quad(integrand, -3, 3) # , points=5, epsrel = 1e-012


In [45]:
ML_params.x

array([-2.84102497, -2.27973843, -1.64825577,  0.4549075 ,  0.46054998,
        0.44285076])

In [37]:
conf_int_80

{0: 0.453422384060298,
 1: 0.44503169910347373,
 2: 0.4683908687360189,
 3: 0.45141393561037013,
 4: 0.44618363256352206,
 5: 0.44415721103658323,
 6: 0.44331481194637745,
 7: 0.4628632418360703,
 8: 0.46530744587452033,
 9: 0.4562313117040477,
 10: 0.4429591246192418,
 11: 0.4556260130044859,
 12: 0.44662234871363937,
 13: 0.4455458916763938,
 14: 0.45143787197265467,
 15: 0.44913818479071393,
 16: 0.4532612344989133,
 17: 0.4404466686031151,
 18: 0.4424410935124721,
 19: 0.4486848842774288,
 20: 0.4403650463984812,
 21: 0.4511864825504336,
 22: 0.4515029153728467,
 23: 0.4486537719333209,
 24: 0.44891343085754865,
 25: 0.4637299707517066,
 26: 0.4520497164564793,
 27: 0.45729817512461424,
 28: 0.4447669551607765,
 29: 0.4570580732238268,
 30: 0.44485834235071187,
 31: 0.45143969235142756,
 32: 0.4476813539148009,
 33: 0.4504537363371431,
 34: 0.4494412223526513,
 35: 0.44331886712570745,
 36: 0.45840678177523697,
 37: 0.4481480748019521,
 38: 0.43819014659024497,
 39: 0.4476050320234

In [38]:
input_80

{0: ([482, 2916, 14294], [240000, 240000, 240000]),
 1: ([295, 1849, 10183], [240000, 240000, 240000]),
 2: ([382, 2540, 13180], [240000, 240000, 240000]),
 3: ([488, 2785, 13530], [240000, 240000, 240000]),
 4: ([453, 2496, 12572], [240000, 240000, 240000]),
 5: ([631, 3243, 14862], [240000, 240000, 240000]),
 6: ([340, 2119, 11844], [240000, 240000, 240000]),
 7: ([420, 2535, 13055], [240000, 240000, 240000]),
 8: ([306, 1983, 11370], [240000, 240000, 240000]),
 9: ([495, 2734, 14019], [240000, 240000, 240000]),
 10: ([344, 2190, 11815], [240000, 240000, 240000]),
 11: ([340, 2165, 12410], [240000, 240000, 240000]),
 12: ([457, 2722, 13734], [240000, 240000, 240000]),
 13: ([460, 2745, 14605], [240000, 240000, 240000]),
 14: ([392, 2537, 13493], [240000, 240000, 240000]),
 15: ([415, 2468, 13409], [240000, 240000, 240000]),
 16: ([359, 2333, 12775], [240000, 240000, 240000]),
 17: ([525, 2937, 15016], [240000, 240000, 240000]),
 18: ([318, 2082, 11568], [240000, 240000, 240000]),
 19

In [57]:
results_dict_80

{0: array([-2.84357483, -2.30875931, -1.68513601,  0.45342238,  0.4513224 ,
         0.44955707]),
 1: array([-2.82936034, -2.26319063, -1.63409791,  0.4450317 ,  0.4624961 ,
         0.46250516]),
 2: array([-2.82061521, -2.26469155, -1.64336188,  0.46839087,  0.45180446,
         0.44586129]),
 3: array([-2.78732091, -2.23301403, -1.63152514,  0.45141394,  0.46448216,
         0.46135291]),
 4: array([-2.82721258, -2.28410652, -1.66389756,  0.44618363,  0.46556118,
         0.44046602]),
 5: array([-2.8033919 , -2.2674188 , -1.67230133,  0.44415721,  0.45969184,
         0.45358533]),
 6: array([-2.84073003, -2.27084075, -1.62255182,  0.44331481,  0.46099271,
         0.46512008]),
 7: array([-2.82721413, -2.27157659, -1.64744815,  0.46286324,  0.45618465,
         0.4444902 ]),
 8: array([-2.8358194 , -2.26843944, -1.63573896,  0.46530745,  0.450736  ,
         0.44789477]),
 9: array([-2.76635541, -2.25488269, -1.61691584,  0.45623131,  0.46390631,
         0.46449152]),
 10: array

In [52]:
df = pd.DataFrame(columns=["n_1", "n_2", "n_3", "d_1", "d_2", "d_3", "gamma_1", "gamma_2", "gamma_3", "factor_loading_1", "factor_loading_2", "factor_loading_3"],
                  index=range(100))

for key, value in input_80.items():
    list1, list2 = value
    
    df.loc[key] = pd.Series({"n_1": list2[0], "n_2": list2[1], "n_3": list2[2], "d_1": list1[0], "d_2": list1[1], "d_3": list1[2], 
                             "gamma_1": results_dict_80[key][0], "gamma_2": results_dict_80[key][1], "gamma_3": results_dict_80[key][2],
                             "factor_loading_1": results_dict_80[key][3], "factor_loading_2": results_dict_80[key][4], "factor_loading_3": results_dict_80[key][5]})

In [53]:
df

,n_1,n_2,n_3,d_1,d_2,d_3,gamma_1,gamma_2,gamma_3,factor_loading_1,factor_loading_2,factor_loading_3
0,240000.0,240000.0,240000.0,482.0,2916.0,14294.0,-2.843575,-2.308759,-1.685136,0.453422,0.451322,0.449557
1,240000.0,240000.0,240000.0,295.0,1849.0,10183.0,-2.82936,-2.263191,-1.634098,0.445032,0.462496,0.462505
2,240000.0,240000.0,240000.0,382.0,2540.0,13180.0,-2.820615,-2.264692,-1.643362,0.468391,0.451804,0.445861
3,240000.0,240000.0,240000.0,488.0,2785.0,13530.0,-2.787321,-2.233014,-1.631525,0.451414,0.464482,0.461353
4,240000.0,240000.0,240000.0,453.0,2496.0,12572.0,-2.827213,-2.284107,-1.663898,0.446184,0.465561,0.440466
...,...,...,...,...,...,...,...,...,...,...,...,...
95,240000.0,240000.0,240000.0,489.0,2660.0,13211.0,-2.778186,-2.263781,-1.650797,0.443059,0.458925,0.458249
96,240000.0,240000.0,240000.0,485.0,2771.0,14233.0,-2.875111,-2.31631,-1.690146,0.453582,0.447066,0.447978
97,240000.0,240000.0,240000.0,499.0,3004.0,15026.0,-2.854882,-2.294498,-1.660718,0.444531,0.46465,0.442278
98,240000.0,240000.0,240000.0,497.0,2652.0,13112.0,-2.773975,-2.257909,-1.644024,0.444385,0.467723,0.458867


In [59]:
df.to_csv("data\conf_interval_data_80_diff_w.csv")

In [28]:
%%time
#factor_loading_init = [0.45]
factor_loading_init = [0.45]
gamma_list_init = [-2.9, -2.3, -1.6]
# gamma_list_init = [-2.87, -2.3, -1.6]
# gamma_list_init = [-2.05, -1.85, -1.6, -1.6, -1.6]
ML_params = parameter_estimation(d_g_list, n_g_list, factor_loading_init, gamma_list_init)
#print("Estimated parameters for speculative grade: ", ML_params)

Optimization terminated successfully.
         Current function value: 18.651324
         Iterations: 87
         Function evaluations: 170
CPU times: total: 9.33 s
Wall time: 10.3 s


In [29]:
ML_params.message

'Optimization terminated successfully.'

In [30]:
#real gamma: [-2.0, -1.8, -1.3]

In [31]:
ML_params.x

array([-2.7616316 , -2.24061306, -1.64078253,  0.46009378])

In [24]:
ML_params.x

array([-2.73231316, -2.2221453 , -1.62828771,  0.4619247 ])

In [22]:
-np.log(calculate_my_likelihood_arr(d_g_list, n_g_list, p_g, prob_dens_func, ML_params.x[3], ML_params.x[0:3]))

18.65880578771864

In [23]:
-np.log(likelihood_mc(ML_params.x[3], ML_params.x[0:3], d_g_list, n_g_list, p_g, num_of_simulations=1000000))

19.640462340924813

In [24]:
-np.log(likelihood_mc(factor_loading_init, gamma_list_init, d_g_list, n_g_list, p_g, num_of_simulations=1000000))

37.48633421546001

In [25]:
-np.log(likelihood_mc(factor_loading_list[0], gamma_list, d_g_list, n_g_list, p_g, num_of_simulations=1000000))

47.0720856257602

In [26]:
-np.log(calculate_my_likelihood_arr(d_g_list, n_g_list, p_g, prob_dens_func, factor_loading_list, gamma_list))

51.15128460268474

In [27]:
def w_calc_func(a, b):
    return -b/ np.sqrt(b**2 + 1)

def gamma_calc_func(a, b):
    return a * np.sqrt(1 - w_calc_func(a, b)**2)

def a_calc_func(w, gamma):
    return gamma / np.sqrt(1 - w**2)

def b_calc_func(w, gamma):
    return -w / np.sqrt(1 - w**2)

def calculate_variable_changed_likelihood_arr(d_g_arr, n_g_arr, p_g, prob_dens_func, a, b):
    integrand = lambda x: np.prod(binom.pmf(d_g_arr, n_g_arr, norm.cdf(a*x+b))) * prob_dens_func(x)

    result, _ = quad(integrand, -3, 3)

    return result

In [28]:
a = [a_calc_func(factor_loading_list[i], gamma_list[i]) for i in range(3)]
b = [b_calc_func(factor_loading_list[i], gamma_list[i]) for i in range(3)]
a, b

([-3.2473765635439547, -2.5755055503969295, -1.7916560350587338],
 [-0.5039032598602688, -0.5039032598602688, -0.5039032598602688])

In [29]:
%%time
# Different gamma and same factor loading parameter

# MLE condition and initial guess
initial_guess = np.array([-3.247, -2.576, -1.792, -0.50])
bounds = [(-5, 5), (-5, 5), (-5, 5), (-5, 5)]

# Function to be minimized in weight parameter
objective_function = lambda params: -np.log(calculate_variable_changed_likelihood_arr(d_g_list, n_g_list, p_g, norm.pdf, 
                                                                 np.repeat(params[3], 3), 
                                                                 params[0:3]))

result = minimize(objective_function,
                  initial_guess,
                  method="Nelder-Mead",
                  bounds=bounds,
                  options={
                      'disp': True})
# Method can be Nelder-Mead or Powell

# The optimal weight parameter
optimal_weight = result.x
print(f"The optimal weight parameter is {optimal_weight}")
print(result.message)

Optimization terminated successfully.
         Current function value: 18.658806
         Iterations: 80
         Function evaluations: 160
The optimal weight parameter is [-3.21409413 -2.57187844 -1.87304655 -0.50822791]
Optimization terminated successfully.
CPU times: total: 4.52 s
Wall time: 4.51 s


In [30]:
result.x

array([-3.21409413, -2.57187844, -1.87304655, -0.50822791])

In [31]:
b_result = result.x[3]
a_result = [result.x[i] for i in range(3)]
w_result = [w_calc_func(a_result[i], b_result) for i in range(3)]
gamma_result = [gamma_calc_func(a_result[i], b_result) for i in range(3)]

w_result, gamma_result

([0.4530719424852357, 0.4530719424852357, 0.4530719424852357],
 [-2.8652812090681987, -2.2927626474283787, -1.6697722189139166])

In [32]:
import random

In [33]:
def expected_value_of_function_monte_carlo(f, num_samples=100000):
    """
    Calculates the expected value of a given function f(x) where x follows the standard normal distribution.

    Parameters:
        f (callable): The function f(x) to calculate the expected value.
        num_samples (int): The number of samples to use in the Monte Carlo simulation.

    Returns:
        float: The estimated expected value of the function.
    """
    # Generate random samples from the standard normal distribution
    random_samples = np.random.normal(size=num_samples)

    # Calculate the expected value using the Monte Carlo simulation
    estimated_expected_value = np.mean(f(random_samples))

    return estimated_expected_value

In [34]:
def likelihood_mc(w_g, gamma_g, d_g, n_g, p_g, num_of_simulations):
    # likelihood function
    likelihood_function = lambda x: np.prod([stats.binom.pmf(d, n, p_g(x, w_g=w_g, gamma_g=gamma)) for d, n, gamma in zip(d_g, n_g, gamma_g)], axis=0)

    return expected_value_of_function_monte_carlo(likelihood_function, num_of_simulations)

In [35]:
# Monte Carlo
def monte_carlo_MLE(d_g, n_g, p_g, w_initial, gamma_initial, bounds, num_of_simulations, seed=None):
    """
    Estimate w_g and gamma_g using the maximum likelihood estimation method.
    Parameters:
        d_g (list): number of defaults.
        n_g (list): number of obligors.
        p_g (callable): The p_g function representing the probability density function.
        w_initial (float): Initial guess for w_g.
        gamma_initial (float): Initial guess for gamma_g.
        bounds (list): Bounds for the minimization algorithm.
        num_of_simulations (int): Number of simulations for the Monte Carlo method.
        seed (int): Seed for the random number generator.
    Returns:
        tuple: Estimated w_g and gamma_g.
    """
    if seed is None:
        seed = random.randint(1, 10000)

    np.random.seed(seed)

    # objective function
    objective_function = lambda params: -np.log(likelihood_mc(params[0], params[1:], d_g, n_g, p_g, num_of_simulations))

    initial_guess = w_initial + gamma_initial
    print(initial_guess)
    print(bounds)

    result = minimize(objective_function, initial_guess, method='Nelder-Mead', bounds=bounds)
    #result = minimize(objective_function, initial_guess, bounds=bounds)

    # The found value of w_g and gamma_g
    return result

In [36]:
%%time
Monte_carlo_params = monte_carlo_MLE(d_g_list, n_g_list, p_g, factor_loading_init, gamma_list_init,
                                     [(-1, 1), (-5, 5), (-5, 5), (-5, 5)], num_of_simulations=10000)

[0.44, -2.87, -2.3, -1.6]
[(-1, 1), (-5, 5), (-5, 5), (-5, 5)]
CPU times: total: 7.89 s
Wall time: 8.03 s


In [37]:
Monte_carlo_params

       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: 17.66211480811598
             x: [ 4.453e-01 -2.824e+00 -2.296e+00 -1.654e+00]
           nit: 307
          nfev: 800
 final_simplex: (array([[ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00],
                       [ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00],
                       ...,
                       [ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00],
                       [ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00]]), array([ 1.766e+01,  1.778e+01,  1.785e+01,  1.786e+01,
                        1.789e+01]))

In [38]:
Monte_carlo_params.x

array([ 0.44530394, -2.82379343, -2.29586094, -1.65412611])